# Simulation-Based Inference for Methylation Inheritance Data

In [1]:
#Import necessary libraries

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm
# from sbi import analysis as analysis
import torch
from sbi.analysis import pairplot
from sbi.inference import NPE
# from sbi.inference import SNPE
# from sbi.inference import simulate_for_sbi
from sbi.utils import BoxUniform
from sbi.utils.user_input_checks import (
    check_sbi_inputs,
    process_prior,
    process_simulator,
)


## Definition of prior and calculation of posterior

In [ ]:
#Train inference of posterior distribution

num_dim = 3
#Create prior box
priormin = [0.001,0.001,1]
priormax = [0.999,3,1000]
prior = BoxUniform(low=torch.as_tensor(priormin), high=torch.as_tensor(priormax))
# Check prior, return PyTorch prior.
prior, num_parameters, prior_returns_numpy = process_prior(prior)

# Check simulator, returns PyTorch simulator able to simulate batches.
simulatorn = process_simulator(simulator_s, prior, prior_returns_numpy)

# Consistency check after making ready for sbi.
check_sbi_inputs(simulatorn, prior)

#Sample parameters from priors
num_simulations = 10000
theta = prior.sample((num_simulations,))
x = simulatorn(theta)
print("theta.shape", theta.shape)
print("x.shape", x.shape)

#Inference step
inference = NPE(prior=prior, density_estimator="nsf")

#Train
density_estimator = inference.append_simulations(theta,x).train()

#Create posterior
posterior = inference.build_posterior(density_estimator)

print(posterior) # prints how the posterior was trained


In [ ]:
#Save inferred posterior
import pickle

f = open('./nnsbi.pckl', 'wb')
pickle.dump(posterior, f)
f.close()

In [ ]:
#Load inferred posterior
import pickle

f = open('./nnsbi.pckl', 'rb')
posterior = pickle.load(f)
f.close()

## Read files from CSV and fit parameters

In [ ]:
import pandas as pd

#Load Amir Dataset

readfile = pd.read_csv("/Users/ciarchi/Nextcloud/MethylationTransm/Julia/data.csv")

In [ ]:
#Create necessary containters for data

data_vars = readfile["daughter.var"]
data_drift = readfile["meth.drift"]
meth_parent = readfile["methyPercent.parent"]*0.01
groups = readfile["group"]
skip = 7
size = int(len(data_vars)/(skip)*7)
sizet = len(data_vars)
sim_vars = np.zeros(size)
sim_drift = np.zeros(size)
sim_noise = np.zeros(size)
sim_parent = np.zeros(size)
sim_group = np.zeros(size)

In [ ]:
#Fit alpha values to Amir data

sel = 0
percent = int(size/100) #Counter for progress
print("One Percent: " + str(percent) + "\n") #Check percentage in datapoints
act_percent = 0
progress_bar = "["+" "*99+"]"
print("Progress: " + "\n" + progress_bar, end = "\r")
for i in range(0,sizet-skip,skip):
    #print(i)
    T = 10
    dt = 0.01
    Nt = int(T/dt)
    m0 = meth_parent[i]
    v0 = data_vars[i]
    mf = m0+data_drift[i]
    group = groups[i]
    daughters = [0,0,0,0,0,0,0]
    samplest = posterior.sample((1,), x=[mf,v0],show_progress_bars=False)
    noise_s = float(torch.median(samplest[:,1]))
    m01 = m0
    if m01 < 0.5 and m01 > 0.25:
        m01 -= 0.1
    elif m01 > 0.5 and m01 < 0.75:
        m01 += 0.1
    #print("Starting values: ",noise_s, " ", m01, " ", m02)
    for p in range(0,7):
        ms1 = m01
        # ms2 = m02
        ms1 = trajectory(Nt,dt,ms1,noise_s)
        # ms2 = trajectory(Nt,dt,ms2,noise_s)
        daughters[p] = ms1#0.5*(ms1+ms2)
    vart = np.var(daughters)
    driftt = np.mean(daughters)-m0
    #print("Confront: ", vart, " ", v0, " ", mf, " ", np.mean(daughters))
    # if abs(vart-v0)>v0*0.1:
    #     print("Not fit well")
    #print(vart, " ", v0, " ", noise_s, "\n")
    for p in range(0,7):
        sim_vars[sel+p] = vart
        sim_drift[sel+p] = driftt
        sim_noise[sel+p] = noise_s
        sim_parent[sel+p] = m0
        sim_group[sel+p] = group
    sel+=7
    current_per = sel
    # print(current_per)
    if current_per >= (act_percent+1)*percent:
        act_percent+=1
        progress_bar_t = "["+"|"*act_percent+" "*(99-act_percent)+"]"
        print(progress_bar_t, end ="\r")
      

In [ ]:
datatosave = np.array([sim_vars, sim_drift, sim_parent, sim_noise, sim_group]).transpose()
dataset = pd.DataFrame({'variances': datatosave[:, 0], 'methdrift': datatosave[:, 1],'methylationParent': datatosave[:, 2], 'bin_var': datatosave[:, 3], 'group': datatosave[:, 4]})
dataset.to_csv('/Users/ciarchi/Desktop/Acadm/MPI/CorrMethyMaintenance/daughterfitsbi.csv')

## Definition of functions and structures

In [ ]:
import numpy as np
import torch

def trajectory(Nt,dt,ms,p,cov):
    m_ = ms
    for i in range(0,Nt):
        m_ = m_ + dt*-((2*m_-1)-np.tanh(p*(2*m_-1))) + np.sqrt(dt/cov)*np.sqrt(abs(1-(2*m_-1)*np.tanh(p*(2*m_-1))))*np.random.normal()
        if m_ < 0:
            m_ = -m_
        elif m_ > 1:
            m_ = 2-m_
    return m_
    
    
def simulator(params):
    T = 10
    dt = 0.01
    Nt = int(T/dt)
    daughters = [0,0,0,0,0,0,0]
    m10 = params[0]
    noiset = params[1]
    m1 = m10
    m20 = m10
    m2 = m10
    # if m10 < 0.5 and m10 > 0.25:
    #     m10 -= 0.1
    # elif m10 > 0.5 and m10 < 0.75:
    #     m10 += 0.1
    # if m20 < 0.5 and m20 > 0.25:
    #     m20 -= 0.1
    # elif m20 > 0.5 and m20 < 0.75:
    #     m20 += 0.1
    for p in range(7):
        m1 = trajectory(Nt,dt,m10,noiset,32)
        m2 = trajectory(Nt,dt,m20,noiset,32)
        #print(m1, " ", m2, "\n")
        daughters[p] = (m1+m2)/2
    # print(daughters, " ", noiset)
    return torch.as_tensor([np.mean(daughters),np.var(daughters)])

#Simulate five trajectories at fixed coverage
def simulator_s(params):
    T = 10
    dt = 0.01
    Nt = int(T/dt)
    m10 = params[0]
    noiset = params[1]
    daughters = [0,0,0,0,0]
    cov = params[2]
    m1 = m10
    m20 = m10
    m2 = m10
    for p in range(5):
        m1 = trajectory(Nt,dt,m10,noiset,cov)
        m2 = trajectory(Nt,dt,m20,noiset,cov)
        daughters[p] = (m1+m2)/2
    return torch.as_tensor([np.mean(daughters),np.var(daughters),cov])


## Checks for distributions

In [ ]:
x_obs = [0.97,0.0001,765]
samples = posterior.sample((100,), x=x_obs)
#simulator(np.asarray(samples[0]))

In [ ]:
import torch
torch.var(samples[:,1]).item()

In [ ]:
_ = pairplot(samples,
             limits=[[0, 1], [-2, 2], [0, 1000]],
             figsize=(6, 6),
             labels=[r"$\theta_1$", r"$\theta_2$", r"$\theta_3$"])


## Simulate dynamics for different values of alpha and look at average dynamics

In [ ]:
import matplotlib.cm as cm


#Time points
Nt = 5000
#Time-step
dt = 0.01
#Number of cells
Nc = 5000
#Path where to save figure
save_path = "/Users/ciarchi/Nextcloud/MethylationTransm/Pictures/Simulations/"
name_file = save_path+"alpha_sim_Nc"+str(Nc)+"_Nt"+str(Nt)+"_dt"+str(dt).replace(".","p")+".pdf"
#Initial methylation values
m_init = 0.9
#Vector of alpha values
alphas = [2,1.9,1.8,1.7,1.6,1.5,1.4]
#Vector of cells
cells = [[[0 for _ in range(Nc)] for _ in range(Nt)] for _ in range(len(alphas))]
#Iterate over cells, every cell simulates two alleles for Nt timesteps
for a in range(0,len(alphas)):
    alpha = alphas[a]
    print("Doing ",alpha,"\n")
    for c in range(0,Nc):
        m1 = m_init
        m2 = m_init
        for t in range(0,Nt):
            m1 = m1 + dt*-((2*m1-1)-np.tanh(alpha*(2*m1-1))) + np.sqrt(dt/32)*np.sqrt(abs(1-(2*m1-1)*np.tanh(alpha*(2*m1-1))))*np.random.normal()
            m2 = m2 + dt*-((2*m2-1)-np.tanh(alpha*(2*m2-1))) + np.sqrt(dt/32)*np.sqrt(abs(1-(2*m2-1)*np.tanh(alpha*(2*m1-1))))*np.random.normal()
            if m1 < 0:
                m1 = -m1
            elif m1 > 1:
                m1 = 2-m1
            if m2 < 0:
                m2 = -m2
            elif m1 > 1:
                m2 = 2-m2
            cells[a][t][c] = 0.5*(m1+m2)

#Average over cells
trajectories = [[0 for _ in range(Nt)] for _ in range(len(alphas))]
for a in range(0,len(alphas)):
    for t in range(0,Nt):
        trajectories[a][t] = np.mean(cells[a][t][:])


In [ ]:
#Path where to save figure
save_path = "/Users/ciarchi/Nextcloud/MethylationTransm/Pictures/Simulations/"
name_file = save_path+"alpha_sim_Nc"+str(Nc)+"_Nt"+str(Nt)+"_dt"+str(dt).replace(".","p")+"_short_"+"norm"+".pdf"
#Plot as function of alpha
time_points = np.arange(2000.0*dt, Nt*dt, dt)
alpha_values = alphas
num_alphas = len(alpha_values)
    
if time_points is None:
    time_points = np.arange(Nt)
if alpha_values is None:
    alpha_values = [f'Alpha {i}' for i in range(num_alphas)]

colormap = cm.get_cmap("viridis_r", num_alphas)

fig, ax = plt.subplots(figsize=(3.33333333333, 2.5))
for i in range(num_alphas):
    ax.plot(time_points, trajectories[i][2000:]/trajectories[i][2000], label=alpha_values[i], color=colormap(i / (num_alphas - 1)),linewidth=0.7)

plt.xlabel("Time")
plt.ylabel("Average methylation")
# plt.title("Curves for Different Coordination Strenghts")
plt.legend()
plt.grid(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

if save_path:
        plt.savefig(name_file, format='pdf')

plt.show()

In [ ]:
#Time points
Nt = 50000
#Time-step
dt = 0.01
#Number of cells
Nc = 1000
#Initial methylation values
m_init = 0.9
#Vector of alpha values
alphas = [1.6]
#Vector of cells
cells = [[[0 for _ in range(Nc)] for _ in range(Nt)] for _ in range(len(alphas))]
#Iterate over cells, every cell simulates two alleles for Nt timesteps
for a in range(0,len(alphas)):
    alpha = alphas[a]
    print("Doing ",alpha,"\n")
    for c in range(0,Nc):
        m1 = m_init
        m2 = m_init
        for t in range(0,Nt):
            m1 = m1 + dt*-((2*m1-1)-np.tanh(alpha*(2*m1-1))) + np.sqrt(dt/32)*np.sqrt(abs(1-(2*m1-1)*np.tanh(alpha*(2*m1-1))))*np.random.normal()
            m2 = m2 + dt*-((2*m2-1)-np.tanh(alpha*(2*m2-1))) + np.sqrt(dt/32)*np.sqrt(abs(1-(2*m2-1)*np.tanh(alpha*(2*m1-1))))*np.random.normal()
            if m1 < 0:
                m1 = -m1
            elif m1 > 1:
                m1 = 2-m1
            if m2 < 0:
                m2 = -m2
            elif m1 > 1:
                m2 = 2-m2
            cells[a][t][c] = 0.5*(m1+m2)

#Average over cells
trajectories_n = [[0 for _ in range(Nt)] for _ in range(len(alphas))]
for a in range(0,len(alphas)):
    for t in range(0,Nt):
        trajectories_n[a][t] = np.mean(cells[a][t][:])


In [ ]:
#Plot as function of alpha
time_points = np.arange(1000.0*dt, Nt*dt, dt)
alpha_values = alphas
num_alphas = len(alpha_values)
    
if time_points is None:
    time_points = np.arange(Nt)
if alpha_values is None:
    alpha_values = [f'Alpha {i}' for i in range(num_alphas)]

colormap = cm.get_cmap("viridis_r", num_alphas)

fig, ax = plt.subplots(figsize=(3.33333333333, 2.5))
for i in range(num_alphas):
    ax.plot(time_points, trajectories_n[i][1000:]/trajectories_n[i][1000], label=alpha_values[i],linewidth=0.7)

plt.xlabel("Time")
plt.ylabel("Average methylation")
# plt.title("Curves for Different Coordination Strenghts")
plt.legend()
plt.grid(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.show()

# Read ageing data and infere alpha values

## Read Petkovich files and put into data frame, then infer alpha values

In [ ]:
#Read file

import pandas as pd
import numpy as np

#Read file
readfile = pd.read_csv("/Users/ciarchi/Desktop/TMPAgeing/petkovich.csv")

In [ ]:
#Remove coverages above 1000
readfile.drop(readfile[readfile['cov'] >= 1000].index, inplace = True)
readfile.drop(readfile[readfile['Age'] == "2.5 (at isolation)"].index, inplace = True)
readfile.drop(readfile[pd.to_numeric(readfile['Age']) < 12.0].index, inplace = True)

#Sample random row by start and tissue
g = readfile.groupby(['chr', 'start'])

readfile = readfile[g.ngroup().isin(np.random.choice(g.ngroups, 100000, replace=False))].dropna()

In [ ]:
# Infer alpha for every site

import numpy as np
from tqdm.notebook import tqdm
import math

#Save vectors
average_meth = readfile["avg.meth"].tolist()
average_cov = readfile["avg.cov"].tolist()
variance = readfile["var.meth"].tolist()

#Create vector of alphas
alphas = np.zeros(len(average_meth))

for i in tqdm(range(0,len(average_meth))):
    if math.isnan(variance[i]):
        alphas[i] = 2.0
        continue
    elif variance[i] == 0:
        alphas[i] = 2.0
        continue
    else:
        x_obs = [average_meth[i],variance[i],average_cov[i]]
        alphas[i] = posterior.sample((1,), x=x_obs, show_progress_bars=False)[0,1].item()

#Add column to data frame
readfile["alpha"] = alphas

## Read Stubbs files and put into data frame, then infer alpha for every site

In [34]:
#Read file

import pandas as pd
import numpy as np

#Read file
readfile = pd.read_csv("/Users/ciarchi/Desktop/TMPAgeing/Stubbs.csv")

/var/folders/jn/hzpz5fjx36x_y2ydy2_b66sc0000gp/T/ipykernel_60785/4103929108.py:7: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  readfile = pd.read_csv("/Users/ciarchi/Desktop/TMPAgeing/Stubbs.csv")


In [35]:
import numpy as np

#Remove coverages above 1000
readfile.drop(readfile[readfile['avg.cov'] >= 1000].index, inplace = True)

#Sample random row by start and tissue
g = readfile.groupby(['tissue', 'start'])

readfile = readfile[g.ngroup().isin(np.random.choice(g.ngroups, 1000000, replace=False))].dropna()

In [36]:
# Infer alpha for every site

import numpy as np
from tqdm.notebook import tqdm
import math

#Save vectors
average_meth = readfile["avg.meth"].tolist()
average_cov = readfile["avg.cov"].tolist()
variance = readfile["var.meth"].tolist()

#Create vector of alphas
alphas = np.zeros(len(average_meth))

for i in tqdm(range(0,len(average_meth))):
    if math.isnan(variance[i]):
        alphas[i] = 2.0
        continue
    elif variance[i] == 0:
        alphas[i] = 2.0
        continue
    else:
        x_obs = [average_meth[i],variance[i],average_cov[i]]
        alphas[i] = posterior.sample((1,), x=x_obs, show_progress_bars=False)[0,1].item()

#Add column to data frame
readfile["alpha"] = alphas


  0%|          | 0/2912283 [00:00<?, ?it/s]

In [ ]:
obs = torch.tensor(readfile[['avg.meth','var.meth','avg.cov']].values).to(dtype=torch.float32) 
try_p = posterior.sample_batched((1,),x=obs,  max_sampling_batch_size = 30000)

In [37]:
#Save to .csv
readfile.to_csv('/Users/ciarchi/Desktop/TMPAgeing/stubbs_alpha.csv')